In [3]:
import tensorflow as tf
import keras 
from keras.applications import ResNet152V2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

In [4]:
conv_base = ResNet152V2(weights='imagenet', include_top = False, input_shape=(300,300,3))

234553344/234545216 [==============================] - 17s 0us/step


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [6]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [7]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [8]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [9]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,10,10,2048))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 10, 10, 2048)
(59, 10, 10, 2048)
(59, 10, 10, 2048)
(59, 10, 10, 2048)
(59, 10, 10, 2048)
(59, 10, 10, 2048)
(59, 10, 10, 2048)
(59, 10, 10, 2048)


In [10]:
samples = samples.reshape(nos,10*10*2048)

In [11]:
samples.shape, labels.shape

((472, 204800), (472,))

In [12]:
del X
del y

In [13]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        model = models.Sequential()
        model.add(layers.Dense(256, activation = 'relu', input_dim = 10*10*2048))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 2s 5ms/step - loss: 10.4304 - accuracy: 0.6634 - val_loss: 4.5897 - val_accuracy: 0.7627
Epoch 2/100
413/413 [==============================] - 2s 4ms/step - loss: 5.2114 - accuracy: 0.8378 - val_loss: 3.3170 - val_accuracy: 0.7627
Epoch 3/100
413/413 [==============================] - 2s 4ms/step - loss: 1.4238 - accuracy: 0.9080 - val_loss: 2.0212 - val_accuracy: 0.7966
Epoch 4/100
413/413 [==============================] - 2s 4ms/step - loss: 1.1772 - accuracy: 0.9492 - val_loss: 1.5374 - val_accuracy: 0.8475
Epoch 5/100
413/413 [==============================] - 2s 4ms/step - loss: 0.1638 - accuracy: 0.9661 - val_loss: 2.4384 - val_accuracy: 0.8136
Epoch 6/100
413/413 [==============================] - 2s 4ms/step - loss: 0.7702 - accuracy: 0.9637 - val_loss: 3.0131 - val_accuracy: 0.7797
Epoch 7/100
413/413 [==============================] - 2s 4ms/step - loss: 0.1929 - accur

In [14]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.8390804597701149 0.7795977011494252


Training on whole Training Dataset

In [20]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 10*10*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [21]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [22]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [23]:
datagentest.fit(xtest)

In [24]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,10,10,2048))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 10, 10, 2048)


In [25]:
test_samples = test_samples.reshape(testnos,10*10*2048)

In [26]:
historytest = model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 3s 6ms/step - loss: 12.3021 - accuracy: 0.6843 - val_loss: 7.5187 - val_accuracy: 0.8098
Epoch 2/100
472/472 [==============================] - 3s 6ms/step - loss: 2.5905 - accuracy: 0.8623 - val_loss: 5.3068 - val_accuracy: 0.8370
Epoch 3/100
472/472 [==============================] - 3s 6ms/step - loss: 2.0789 - accuracy: 0.9089 - val_loss: 6.8178 - val_accuracy: 0.8098
Epoch 4/100
472/472 [==============================] - 3s 6ms/step - loss: 1.6179 - accuracy: 0.9428 - val_loss: 6.3786 - val_accuracy: 0.7446
Epoch 5/100
472/472 [==============================] - 3s 6ms/step - loss: 0.3461 - accuracy: 0.9682 - val_loss: 7.0639 - val_accuracy: 0.7264
Epoch 6/100
472/472 [==============================] - 3s 6ms/step - loss: 0.4495 - accuracy: 0.9788 - val_loss: 4.7738 - val_accuracy: 0.7953
Epoch 7/100
472/472 [==============================] - 3s 6ms/step - loss: 0.1894 - accuracy: 0

In [27]:
test_preds = model.predict(test_samples, batch_size = 20)

In [28]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [29]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.6779661016949152, 0.8701825557809331)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/